In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics import r2_score

In [2]:
meta = pd.read_csv("../input/meta_open.csv", index_col='uid', parse_dates=["datastart","dataend"], dayfirst=True)
temporal = pd.read_csv("../input/temp_open_utc_complete.csv", index_col='timestamp', parse_dates=True).tz_localize('utc')

In [3]:
# All models types
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import GradientBoostingRegressor
from  sklearn.linear_model import HuberRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import TheilSenRegressor
    
# Make array of models. Each model is an array of two elements.
# First element is a model-name, second is a model itself
models = [#['RandomForestRegressor', RandomForestRegressor(n_estimators = 1000, random_state = 42)],
#['AdaBoostRegressor', AdaBoostRegressor(n_estimators = 1000, random_state = 42)],
#['BaggingRegressor', BaggingRegressor(n_estimators = 1000, random_state = 42)],
#['DecisionTreeRegressor', DecisionTreeRegressor(random_state = 42)],
['DummyRegressor', DummyRegressor()],
#['ExtraTreeRegressor', ExtraTreeRegressor(random_state = 42)],
#['ExtraTreesRegressor', ExtraTreesRegressor(n_estimators = 1000, random_state = 42)],
#['GaussianProcessRegressor', GaussianProcessRegressor(random_state = 42)],
#['GradientBoostingRegressor', GradientBoostingRegressor(n_estimators = 1000, random_state = 42)],
#['HuberRegressor', HuberRegressor()],
#['KNeighborsRegressor', KNeighborsRegressor()],
#['MLPRegressor', MLPRegressor(random_state = 42)],
#['PassiveAggressiveRegressor', PassiveAggressiveRegressor(random_state = 42)],
#['RANSACRegressor', RANSACRegressor(random_state = 42)],
#['SGDRegressor', SGDRegressor(random_state = 42)],
#['TheilSenRegressor', TheilSenRegressor(random_state = 42)]
]

In [7]:
# Produce file with metrics(MAPE, NMBE, CVRSME, RSQUARED) based on provided model
# Results will be saved as modelName_metrics.csv
def createMetrics(modelName, model, buildingtype):
    buildingnames = temporal.columns[temporal.columns.str.contains(buildingtype)]
    print('\n\n' + modelName + '-' + buildingtype + '\n_____________')
    
    # create empty variable to aggregate schedule data
    dailyshedule = pd.DataFrame([],columns=["date", "status"])
    # append all schedule data into one data frame
    for i in range(1,18):
        singleshedule = pd.read_csv(os.path.join("../input/schedule" + str(i) + ".csv"), parse_dates=True, na_values='-9999', names = ["date", "status"])
        dailyshedule = dailyshedule.append(singleshedule, ignore_index=True)
    dailyshedule['date'] = pd.to_datetime(dailyshedule['date'])
    dailyshedule = dailyshedule.sort_values(by='date')
    dailyshedule = dailyshedule.drop_duplicates(subset='date', keep="first")
    
    for singlebuilding in buildingnames[:3]:
        print("Modelling: " + singlebuilding)
        # Get Data
        single_timezone = meta.T[singlebuilding].timezone
        single_start = meta.T[singlebuilding].datastart
        single_end = meta.T[singlebuilding].dataend
        single_building_data = pd.DataFrame(temporal[singlebuilding].tz_convert(single_timezone).truncate(before=single_start,after=single_end))
        
        # select schedule data according to a single building(start/end date) 
        mask = (dailyshedule['date'] > single_start) & (dailyshedule['date'] <= single_end)
        # resample daily data to hourly
        hourlyshedule = dailyshedule[mask].set_index('date').resample('H').pad()

        
        # Split into Training and Testing
        trainingdata = single_building_data[single_building_data.index.month.isin(["1","2","3","5","6","7","9","10","11"])]
        testdata = single_building_data[single_building_data.index.month.isin(["4","8","12"])]
        
        # Split into Training and Testing
        trainingshedule = hourlyshedule[hourlyshedule.index.month.isin(["1","2","3","5","6","7","9","10","11"])]
        testshedule = hourlyshedule[hourlyshedule.index.month.isin(["4","8","12"])]
        
        # Get weather file
        weatherfilename = meta.T[singlebuilding].newweatherfilename
        print("Weatherfile: "+weatherfilename)
        weather = pd.read_csv(os.path.join("../input/",weatherfilename),index_col='timestamp', parse_dates=True, na_values='-9999')
        weather = weather.tz_localize(single_timezone, ambiguous = 'infer')
        
        outdoor_temp = pd.DataFrame(weather[[col for col in weather.columns if 'Temperature' in col]]).resample("H").mean()
        outdoor_temp = outdoor_temp.reindex(pd.DatetimeIndex(start=outdoor_temp.index[0], periods=len(single_building_data), freq="H")).fillna(method='ffill').fillna(method='bfill')
        # Add humidity
        humidity = pd.DataFrame(weather[[col for col in weather.columns if 'Humidity' in col]]).resample("H").mean()
        humidity = humidity.reindex(pd.DatetimeIndex(start=humidity.index[0], periods=len(single_building_data), freq="H")).fillna(method='ffill').fillna(method='bfill')
        # Create training data array
        train_features = np.array(pd.concat([pd.get_dummies(trainingdata.index.hour),
                                             pd.get_dummies(trainingdata.index.dayofweek),
                                             pd.get_dummies(trainingshedule.index.dayofweek),
                   pd.Series(outdoor_temp[outdoor_temp.index.month.isin(["1","2","3","5","6","7","9","10","11"])].TemperatureC.values),
                   pd.Series(humidity[humidity.index.month.isin(["1","2","3","5","6","7","9","10","11"])].Humidity.values)], axis=1))
        train_labels = np.array(trainingdata[singlebuilding].values)

        # Create test data array
        test_features = np.array(pd.concat([pd.get_dummies(testdata.index.hour),
                                             pd.get_dummies(testdata.index.dayofweek),
                                             pd.get_dummies(testshedule.index.dayofweek),
                   pd.Series(outdoor_temp[outdoor_temp.index.month.isin(["4","8","12"])].TemperatureC.values),
                   pd.Series(humidity[humidity.index.month.isin(["4","8","12"])].Humidity.values)], axis=1))
        test_labels = np.array(testdata[singlebuilding].values)


        # Train the model on training data
        model.fit(train_features, train_labels);
        # Use the forest's predict method on the test data
        predictions = model.predict(test_features)

        # Calculate the absolute errors
        errors = abs(predictions - test_labels)
        # Calculate mean absolute percentage error (MAPE) and add to list
        MAPE = 100 * np.mean((errors / test_labels))
        NMBE = 100 * (sum(test_labels - predictions) / (pd.Series(test_labels).count() * np.mean(test_labels)))
        CVRSME = 100 * ((sum((test_labels - predictions)**2) / (pd.Series(test_labels).count()-1))**(0.5)) / np.mean(test_labels)
        RSQUARED = r2_score(test_labels, predictions)

        print("MAPE: "+str(MAPE))
        print("NMBE: "+str(NMBE))
        print("CVRSME: "+str(CVRSME))
        print("R SQUARED: "+str(RSQUARED))

        MAPE_data[singlebuilding] = MAPE
        NMBE_data[singlebuilding] = NMBE
        CVRSME_data[singlebuilding] = CVRSME
        RSQUARED_data[singlebuilding] = RSQUARED

        metrics = pd.DataFrame([MAPE_data, NMBE_data, CVRSME_data, RSQUARED_data]).T
        metrics.columns = ["MAPE", "NMBE", "CVRSME", "RSQUARED"]
        metrics.to_csv('../results/' + modelName + '_metrics_' + buildingtype + '.csv')


In [8]:
for elem in models:
    # Go over all building types
    buildingtypes = ['Office', 'PrimClass', 'UnivClass', 'UnivDorm', 'UnivLab']
    for buildingtype in buildingtypes:
        # clear values
        MAPE_data = {}
        RSQUARED_data = {}
        NMBE_data = {}
        CVRSME_data = {}
        # modelName, model, buildingtype
        createMetrics(elem[0], elem[1], buildingtype)
    



DummyRegressor-Office
_____________
Modelling: Office_Cristina
Weatherfile: weather2.csv
MAPE: 31.48652791375246
NMBE: -1.4879135920651274
CVRSME: 31.649425974673346
R SQUARED: -0.0022160616661179855
Modelling: Office_Jesus
Weatherfile: weather1.csv
MAPE: 147.472717445552
NMBE: 9.316838322023584
CVRSME: 39.557826365804324
R SQUARED: -0.05875781584640505
Modelling: Office_Jett
Weatherfile: weather1.csv
MAPE: 185.0037145890515
NMBE: -7.02307076064646
CVRSME: 72.05239208060017
R SQUARED: -0.009596254286377004


DummyRegressor-PrimClass
_____________
Modelling: PrimClass_Jolie
Weatherfile: weather1.csv
MAPE: 410.1061480747455
NMBE: -37.805083502964955
CVRSME: 134.59424479383853
R SQUARED: -0.08569425317096568
Modelling: PrimClass_Jaylin
Weatherfile: weather1.csv
MAPE: 253.54604736371846
NMBE: -35.918194856392994
CVRSME: 125.55364317163284
R SQUARED: -0.08917984166824922
Modelling: PrimClass_Uma
Weatherfile: weather10.csv
MAPE: 74.49973459733478
NMBE: -6.1364720503583055
CVRSME: 71.402322